### **Gussian Elimination**

In [9]:
import numpy as np

class GaussianEliminator:
    def __init__(self, A, b):
        self.A = A.astype(float).copy() # Matrix A
        self.b = b.astype(float).copy() # vector b
        self.n = self.A.shape[0]
        self.steps = []

    def eliminate(self):
        """
        Perform Gaussian elimination and convert [A|b] to Row Echelon Form (REF).
        """
        for i in range(self.n):
            
            # Handle zero pivot
            if np.isclose(self.A[i, i], 0):
                swapped = False
                for j in range(i + 1, self.n):
                    if not np.isclose(self.A[j, i], 0):
                        self._swap_rows(i, j)
                        self.steps.append(f"Swapped row {i} with row {j} (pivot was zero).")
                        swapped = True
                        break
                if not swapped:
                    self.steps.append(f"Column {i} pivot is zero and unswappable. Skipping.")
                    continue

            pivot = self.A[i, i] 

            # Elimination
            for j in range(i + 1, self.n):
                multiplier = self.A[j, i] / pivot # multiplier
                self.A[j] -= multiplier * self.A[i] # eliminate an entry in A
                self.b[j] -= multiplier * self.b[i] # eliminate b
                self.steps.append(f"R{j+1} → R{j+1} - ({multiplier:.2f}) × R{i+1}") # steps

    def _swap_rows(self, i, j):
        """Used if there's a zero in pivot position"""
        self.A[[i, j]] = self.A[[j, i]]
        self.b[[i, j]] = self.b[[j, i]]

    def get_augmented_matrix(self):
        """Transform A and b to augmented matrix"""
        return np.column_stack((self.A, self.b))

    def print_steps(self):
        print("Steps:")
        for step in self.steps:
            print(step)

    def print_result(self):
        print("Row Echelon Form [A|b]:")
        print(self.get_augmented_matrix())



A = np.array([[1, 2, 3],
              [2, 5, 2],
              [6, -3, 1]])

b = np.array([[6],
              [4],
              [2]])
eliminator = GaussianEliminator(A, b)
eliminator.eliminate()
eliminator.print_steps()
eliminator.print_result()

Steps:
R2 → R2 - (2.00) × R1
R3 → R3 - (6.00) × R1
R3 → R3 - (-15.00) × R2
Row Echelon Form [A|b]:
[[   1.    2.    3.    6.]
 [   0.    1.   -4.   -8.]
 [   0.    0.  -77. -154.]]


### **Solution Type Detection**

In [6]:
import numpy as np

def detect_solution_type(A, b):
    """
    Detects the type of solution for the linear system Ax = b
    using the rank of A and the augmented matrix [A|b].
    """
    A = A.astype(float) # Matrix A
    b = b.reshape(-1, 1).astype(float) # vector b

    augmented = np.column_stack((A, b)) # make Ab augmented
    rank_A = np.linalg.matrix_rank(A) # computes the rank of the matrix A
    rank_aug = np.linalg.matrix_rank(augmented) # computes the rank of the augmented matrix [A|b]
    n = A.shape[1]  # number of variables

    if rank_A == rank_aug == n:
        return "Unique solution"
    elif rank_A == rank_aug < n:
        return "Infinite solutions"
    elif rank_A < rank_aug:
        return "No solution"
    else:
        return "Indeterminate case"

A = np.array([[1, 2, 3],
              [2, 5, 2],
              [6, -3, 1]])

b = np.array([6, 4, 2])

result = detect_solution_type(A, b)
print(result)

Unique solution


### **Back Subtitution**

In [14]:
def back_substitution_with_steps(U, b):
    """
    Perform back substitution on upper triangular matrix Ux = b,
    printing each step.

    Args:
        U (ndarray): Upper triangular matrix (n x n)
        b (ndarray): Right-hand side vector (n x 1)

    Returns:
        x (ndarray): Solution vector (n x 1)
    """
    n = U.shape[0]
    x = np.zeros((n, 1))
    steps = []

    for i in reversed(range(n)):
        sum_ = np.dot(U[i, i+1:], x[i+1:])
        x[i] = (b[i] - sum_) / U[i, i]
        terms = " + ".join([f"{U[i, j]:.2f}·x{j+1}" for j in range(i+1, n)])
        eq = f"x{i+1} = ({b[i][0]:.2f} - ({terms})) / {U[i, i]:.2f}"
        eq = eq if terms else f"x{i+1} = {b[i][0]:.2f} / {U[i, i]:.2f}"
        steps.append(eq + f" = {x[i][0]:.2f}")

    print("Back Substitution Steps:")
    for s in steps:
        print(s)

    return x


def solve_linear_system_verbose(A, b):
    eliminator = GaussianEliminator(A, b)
    eliminator.eliminate()

    U = eliminator.A
    c = eliminator.b
    return back_substitution_with_steps(U, c)


A = np.array([[1, 2, 3],
              [2, 5, 2],
              [6, -3, 1]])

b = np.array([[6],
              [4],
              [2]])
eliminator.print_result()
x = solve_linear_system_verbose(A, b)
print("Solution:\n", x)

Row Echelon Form [A|b]:
[[   1.    2.    3.    6.]
 [   0.    1.   -4.   -8.]
 [   0.    0.  -77. -154.]]
Back Substitution Steps:
x3 = -154.00 / -77.00 = 2.00
x2 = (-8.00 - (-4.00·x3)) / 1.00 = 0.00
x1 = (6.00 - (2.00·x2 + 3.00·x3)) / 1.00 = 0.00
Solution:
 [[0.]
 [0.]
 [2.]]
